In [1]:
import random
from importlib import reload

import torch
from torch.nn.functional import softmax, log_softmax

from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM

from util import mt

In [2]:
model_name = 'EleutherAI/gpt-neo-125M'
model_name = 'facebook/xglm-7.5B'
model_name = 'facebook/xglm-4.5B'
model_name = 'facebook/xglm-2.9B'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).eval().half().cuda()

In [4]:
src = load_dataset("gsarti/flores_101", 'mya')
tgt = load_dataset("gsarti/flores_101", 'eng')
eng = load_dataset("gsarti/flores_101", 'eng')

/gscratch/zlab/ahai/miniconda3/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/mya/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

/gscratch/zlab/ahai/miniconda3/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)
Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/eng/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
src_lang = 'Burmese'
tgt_lang = 'English'

In [6]:
src_points = random.sample(list(src['dev']), k=10)
tgt_points = [ tgt['dev'][point['id']-1] for point in src_points ]

In [55]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [56]:
demo_prompt_ids = mt.make_demo_prompt(src_points, tgt_points, tokenizer)
pmi_demo_prompt_ids = mt.make_demo_prompt(tgt_points, src_points, tokenizer)
uncond_demo_prompt_ids = mt.make_uncond_demo_prompt(tgt_points, tokenizer)

In [12]:
inference_point = src['dev'][idx] 
cond_prompt_ids = uncond_demo_prompt_ids + s_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :")
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {tgt_lang} :")
uncond_prompt_t = torch.LongTensor([[2]]).to(model.device)
print(idx)
print()
print(eng['dev'][idx]['sentence'])
print()
print(src['dev'][idx]['sentence'])
print()
print(tgt['dev'][idx]['sentence'])

NameError: name 'idx' is not defined

In [392]:
inference_point = src['dev'][idx] 
cond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :")
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + s_ids(f"{src_lang} : {tgt_lang} :")
uncond_prompt_t = torch.LongTensor([uncond_demo_prompt_ids]).to(model.device)
print(eng['dev'][idx]['sentence'])
print()
print(src['dev'][idx]['sentence'])
print()
print(tgt['dev'][idx]['sentence'])

When the tenants started sharing what had occurred to them, most of the families involved suddenly realized that Carolyn Wilson of the OHA had stolen their security deposits, and skipped out of town.

Kun vuokralaiset alkoivat kertoa, mitä heille oli tapahtunut, useimmat perheet, joita asia koski, käsittivät pian, että Oaklandin asuntoviraston edustaja Carolyn Wilson varastanut heidän takuutalletuksensa ja paennut kaupungista.

When the tenants started sharing what had occurred to them, most of the families involved suddenly realized that Carolyn Wilson of the OHA had stolen their security deposits, and skipped out of town.


In [ ]:
with torch.no_grad():
  gen_ids = mt.cfg_p_greedy(model, 0.2, cond_prompt_t, uncond_prompt_t, 50, tokenizer=tokenizer, p=0.5)
print()
print(tokenizer.decode(gen_ids[0].tolist()))

In [8]:
idx = -1

In [9]:
idx += 1

In [10]:
inference_point = src['dev'][idx] 
cond_prompt_ids = demo_prompt_ids + mt.xglm_ids(f"{src_lang} : {inference_point['sentence']} {tgt_lang} :", tokenizer)
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + mt.xglm_ids(f"{tgt_lang} :", tokenizer)
uncond_prompt_t = torch.LongTensor([uncond_prompt_ids]).to(model.device)
print(eng['dev'][idx]['sentence'])
print()
print(src['dev'][idx]['sentence'])
print() 
print(tgt['dev'][idx]['sentence'])

On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.

တနင်္လာနေ့တွင် စတန်းဖို့ဒ်တက္ကသိုလ် ဆေးကျောင်းမှ သိပ္ပံပညာရှင်များသည် ဆဲလ်များကို အမျိုးအစားအလိုက် စီစဉ်နိုင်သော ရောဂါခွဲခြားစစ်ဆေးမှု ကိရိယာအသစ် တီထွင်မှုအကြောင်းကို ကြေညာခဲ့သည်- ၎င်းသည် စံနှုန်းမီ မင်စက်ကလေးများဖြင့် ပုံဖော်သည့် ပရင်တာများကို သုံးကာ ထုတ်လုပ်နိုင်သော အလွန်သေးငယ်သည့် တစ်ခုလျှင် U.S. ဆင့်တစ်ပြားသာသာရှိသော ပရင့်လုပ်၍ရသော အပြားလေးဖြစ်ပါသည်။

On Monday, scientists from the Stanford University School of Medicine announced the invention of a new diagnostic tool that can sort cells by type: a tiny printable chip that can be manufactured using standard inkjet printers for possibly about one U.S. cent each.


In [723]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [51]:
with torch.no_grad():
  gen_ids = mt.tp_greedy(model, 1, cond_prompt_t, uncond_prompt_t, 50, tokenizer=tokenizer, p=0.8)
print()
print(tokenizer.decode(gen_ids[0].tolist()))


The pilot was able to identify the pilot of the group leader, thiskae.</s> Burmese : ဂျာမန်လူမျိုး ဂျော့ဂျ် ဂျီဗာသည် ဂျာမန်လူမျိုး ဂျော့ဂျ် 


In [80]:
idx += 1
inference_point = src['dev'][idx] 
cond_prompt_ids = demo_prompt_ids + mt.xglm_ids(f"{inference_point['sentence']} =", tokenizer)
cond_prompt_t = torch.LongTensor([cond_prompt_ids]).to(model.device)
uncond_prompt_ids = demo_prompt_ids + mt.xglm_ids(f"{tgt_lang} :", tokenizer)
uncond_prompt_t = torch.LongTensor([uncond_prompt_ids]).to(model.device)
print(eng['dev'][idx]['sentence'])
print()
print(src['dev'][idx]['sentence'])
print() 
print(tgt['dev'][idx]['sentence'])

At 11:20, the police asked the protesters to move back on to the pavement, stating that they needed to balance the right to protest with the traffic building up.

မနက် ၁၁:၂၀ တွင် ရဲများက ဆန္ဒပြသူများကို လူသွားစင်္ကြံပေါ်သို့ သွားခိုင်းပြီး ဆန္ဒပြခွင့်နှင့် ယာဉ်ကြောပိတ်ဆို့မှုကို ဟန်ချက်ညီအောင်လုပ်ရန်လိုကြောင်း ပြောသွားပါသည်။

At 11:20, the police asked the protesters to move back on to the pavement, stating that they needed to balance the right to protest with the traffic building up.


In [81]:
with torch.no_grad():
  greedy_gen_ids = mt.greedy(model, cond_prompt_t, 50)
print(tokenizer.decode(greedy_gen_ids[0].tolist()))

The police told the crowd to go to the crowded street and try to work out a solution to the traffic jam.</s> သင့်အနေဖြင့် ခရီးသွားလာမှုဆိုင်ရာ စုစုပေါင်းကိုးကားချက်ကို ကမ္ဘာတစ်လွှား သွားလာရန်


In [82]:
with torch.no_grad():
  p_gen_ids = mt.tp_greedy(model, 1, cond_prompt_t, uncond_prompt_t, 50)
print(tokenizer.decode(p_gen_ids[0].tolist()))

The police told protesters to go to the crowded street and try to work out a solution to traffic congestion.</s> ဆန္ဒပြသူများက ဆန္ဒပြရန်အတွက် ယာဉ်ကြောပိတ်ဆို့မှုကို လျှော့ချ


In [83]:
with torch.no_grad():
  gen_ids = torch.LongTensor([mt.cleanup(greedy_gen_ids[0].tolist())]).to(model.device)
  inference_point = src['dev'][idx]
  pmi_prompt_ids = pmi_demo_prompt_ids + mt.xglm_ids(f"{tgt_lang} : {tokenizer.decode(gen_ids[0])} {src_lang} :", tokenizer)
  pmi_prompt_t = torch.LongTensor([pmi_prompt_ids]).to(model.device)
  src_ids = tokenizer(inference_point['sentence']).input_ids
  src_t = torch.LongTensor([src_ids]).to(model.device)
  pmi_check_t = torch.cat([pmi_prompt_t, src_t], dim=1)
  res = model(pmi_check_t)
  lls = log_softmax(res.logits[0, -1-src_t.size(1):-1], dim=-1)
  gold_lls = lls.gather(1, src_t.T)
  pmi = gold_lls.sum().item()
pmi

-136.25

In [79]:
with torch.no_grad():
  gen_ids = torch.LongTensor([mt.cleanup(p_gen_ids[0].tolist())]).to(model.device)
  inference_point = src['dev'][idx]
  pmi_prompt_ids = pmi_demo_prompt_ids + mt.xglm_ids(f"{tgt_lang} : {tokenizer.decode(gen_ids[0])} {src_lang} :", tokenizer)
  pmi_prompt_t = torch.LongTensor([pmi_prompt_ids]).to(model.device)
  src_ids = tokenizer(inference_point['sentence']).input_ids
  src_t = torch.LongTensor([src_ids]).to(model.device)
  pmi_check_t = torch.cat([pmi_prompt_t, src_t], dim=1)
  res = model(pmi_check_t)
  lls = log_softmax(res.logits[0, -1-src_t.size(1):-1], dim=-1)
  gold_lls = lls.gather(1, src_t.T)
  pmi = gold_lls.sum().item()
pmi

-122.4375

In [74]:
gold_lls.squeeze().tolist()

[-9.0625,
 -5.7421875,
 -6.2890625,
 -0.84716796875,
 -0.0008044242858886719,
 -0.447509765625,
 -0.74365234375,
 -3.9296875,
 -4.67578125,
 -0.2032470703125,
 -0.028533935546875,
 -0.053070068359375,
 -0.64990234375,
 -1.8232421875,
 -0.043060302734375,
 -1.5234375,
 -0.5419921875,
 -3.439453125,
 -0.12359619140625,
 -0.00043892860412597656,
 -0.1541748046875,
 -0.043426513671875,
 -1.1962890625,
 -5.03125,
 -0.3974609375,
 -0.0033550262451171875,
 -2.66796875,
 -1.4013671875,
 -0.42529296875,
 -0.286376953125,
 -3.744140625,
 -0.16015625,
 -2.00390625,
 -4.21875,
 -1.505859375,
 -3.0703125,
 -1.5615234375,
 -0.69921875,
 -4.890625,
 -0.140625,
 -0.0004661083221435547,
 -0.86474609375,
 -0.0007886886596679688,
 -0.72021484375,
 -3.505859375,
 -2.0625,
 -2.107421875,
 -0.00010645389556884766,
 -0.00022351741790771484,
 -2.1796875,
 -4.17578125,
 -4.578125,
 -0.0017328262329101562,
 -4.5299530029296875e-06,
 -4.73828125,
 -4.75390625,
 -1.4541015625,
 -0.68701171875,
 -0.10772705078125,

In [57]:
tokenizer.decode(demo_prompt_ids)

'</s> မြို့၏အခြားလမ်းကြောင်းဖြစ်သော မြို့ပတ်လမ်းကျယ်ပေါ်တွင် အလွန်အမင်းယာဉ်ကြောပိတ်ဆို့နေခြင်းမရှိကြောင်း သတင်းရထားပါသည်။ = There were no heavy traffic delays reported on the beltway, the city\'s alternate route.</s> ဓားသွားကို တစ်ဖက်ခြမ်း သို့မဟုတ် အခြားဖက်ခြမ်းသို့ စောင်းထားသဖြင့် ဓားသွား၏အောက်ခြေမှာ အနည်းငယ်ကွေးနေသောကြောင့်၊ ရေခဲနှင့်ထိခတ်မိသောအစွန်းဘက်နေရာသည်လည်း ကွေးနေပါသည်။ = Because the bottom of the blade is slightly curved, as the blade tilts to one side or the other, the edge which is in contact with the ice also curves.</s> သင့်တွင် ခရီးသွားလာချိန်အပိုရှိပါက အာဖရိကကသို့ သွားရောက်ရန်အတွက် ခရီးသွားလာမှုဆိုင်ရာ စုစုပေါင်းကိုးကားချက်ကို ကမ္ဘာတစ်လွှား သွားလာရန် ကိုးကားချက်နှင့် နှိုင်းယှဉ်ကြည့်ပါ။ = If you have additional travel time, check to see how your total fare quote to Africa compares with a round-the-world fare.</s> မနက်အစောပိုင်းမှာ အီတလီရှိ ကိုဗာစီယာနိုတွင် ရာသီအကြိုလေ့ကျင့်မှု ပြုလုပ်ရာတွင် Jarque ပါဝင်ခဲ့သည်။ သူက ဘိုလိုနီးယားနှင့် တနင်္ဂနွေနေ့တွင် စီစဉ်ထားသည့် ပြိုင်ပ

In [85]:
temp = 1
alpha = 1
eps = torch.finfo(torch.float32).eps
with torch.no_grad():
  c_lls = log_softmax(model(cond_prompt_t).logits[0][-1] / temp, dim=0)
  u_lls = log_softmax(model(uncond_prompt_t).logits[0][-1] / temp, dim=0)
  pmi = c_lls - u_lls
  pmi = pmi.long()
  pmi_dist = pmi - pmi.min() + eps
  pmi_dist = pmi_dist / pmi_dist.sum()
  pmi_lls = pmi_dist.log()
#   pmi.div_(pmi.sum())
#   apmi = alpha*pmi + (1-alpha)*torch.ones_like(pmi).div_(pmi.size(0))
# tokenizer.decode(torch.topk(apmi, 10).indices)
torch.topk(pmi_lls, 100, largest=False)

torch.return_types.topk(
values=tensor([-31.4399, -31.4399, -14.8043, -14.8043, -14.8043, -14.8043, -14.3989,
        -14.3989, -14.3989, -14.3989, -14.3989, -14.3989, -14.3989, -14.1112,
        -14.1112, -14.1112, -14.1112, -14.1112, -14.1112, -14.1112, -14.1112,
        -14.1112, -14.1112, -14.1112, -14.1112, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880,
        -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.8880, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057, -13.7057,
        -13.7057, -13.7057, -13.

In [78]:
torch.Tensor([1.1160e-06]).log()

tensor([-13.7058])

In [1989]:
with torch.no_grad():
  print(torch.topk(log_softmax(model(cond_prompt_t).logits[0][-1] / temp), 10))

torch.return_types.topk(
values=tensor([-0.6235, -0.8735, -5.3750, -5.5000, -6.0000, -6.2500, -6.2500, -6.6250,
        -7.0000, -7.2500], device='cuda:0', dtype=torch.float16),
indices=tensor([   268, 139910,     46,    984,    345,  21992, 102595,     32,   2686,
           826], device='cuda:0'))


/tmp/ipykernel_52376/1101773222.py:2: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.topk(log_softmax(model(cond_prompt_t).logits[0][-1] / temp), 10))


In [775]:
prompt_ids = torch.LongTensor([tokenizer('translate into english, 中文： 谢谢你！').input_ids[:-1]]).to(model.device)

In [232]:
prompt_ids = torch.LongTensor([tokenize('中文： 谢谢你! english:')]).to(model.device)                          

In [233]:
prompt_ids.tolist()

[[6, 19773, 13, 6, 13305, 13305, 710, 35, 83763, 13]]

In [351]:
model.config

XGLMConfig {
  "_name_or_path": "facebook/xglm-2.9B",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "architectures": [
    "XGLMForCausalLM"
  ],
  "attention_dropout": 0.1,
  "attention_heads": 16,
  "bos_token_id": 0,
  "d_model": 2048,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "eos_token_id": 2,
  "ffn_dim": 8192,
  "init_std": 0.02,
  "layerdrop": 0.0,
  "max_position_embeddings": 2048,
  "model_type": "xglm",
  "num_layers": 48,
  "pad_token_id": 1,
  "scale_embedding": true,
  "transformers_version": "4.22.2",
  "use_cache": true,
  "vocab_size": 256008
}

In [275]:
with torch.no_grad():
  gen_ids = mt.sample(model, prompt_ids, 200)

TypeError: expected Tensor as element 0 in argument 0, but got list

In [309]:
tokenizer.decode(gen_ids[0].tolist())

'Leading researchers say that the test can detect cancer, tuberculosis, HIV and malaria in patients who live in low-income countries, where for example, the survival rate of breast cancer is half that of the richer countries.</s> Finnish.</s> Finnish : Hän sanoi, että hän sanoi, että hän sanoi, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että, että,'

In [231]:
print(tokenizer.decode(prompt_ids[0]))

中文: 谢谢你! | english:


In [14]:
inference_point['sentence']

'Stanfordin yliopiston lääketieteen laitoksen tutkijat ilmoittivat maanantaina uuden diagnostiikkatyökalun keksimisestä: solut tyypin mukaan lajitteleva pienenpieni tulostettava siru, joka voidaan valmistaa normaaleilla mustesuihkutulostimilla mahdollisesti noin yhden Yhdysvaltain sentin kappalehintaan.'

In [209]:
s_ids('')

[]

In [219]:
def tokenize(s):
  ids = []
  i = 0
  t = 0
  while t < len(s):
    c = s[t]
    if c == '\n':
      ids += s_ids(s[i:t]) + [2]
      i = t+1
      t = t+1
    else:
      t += 1
  ids += s_ids(s[i:])
  return ids

In [12]:
vocab = tokenizer.get_vocab()
tokens = list(sorted(tokenizer.get_vocab(), key=lambda k: vocab[k]))

In [121]:
tokens[:15]

['<s>',
 '<pad>',
 '</s>',
 '<unk>',
 ',',
 '.',
 '▁',
 's',
 '-',
 'a',
 '▁de',
 '▁a',
 'e',
 ':',
 'i']

In [305]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_auto_class',
 '_batch_encode_plus',
 '_bos_token',
 '_call_one',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad

In [306]:
tokenizer.model_max_length

1000000000000000019884624838656

In [319]:
def make_demo_prompt(src_points, tgt_points, src_name, tgt_name, src, tgt, k, tokenizer):
    demos = []
    for s, t in zip(src_points, tgt_points):
        assert(s['id'] == t['id'])
        demo = f"{src_name} : {s['sentence']} {tgt_name} : {t['sentence']}"
        demos.append(demo)
    demo_prompt_ids = [2]
    for demo in demos:
      demo_prompt_ids.extend(mt.xglm_tokenize(demo, tokenizer))
      demo_prompt_ids.append(2)
    return demo_prompt_ids

In [323]:
src_points = random.sample(list(src['dev']), k=3)
tgt_points = [ tgt['dev'][point['id']-1] for point in src_points ]

In [334]:
reload(mt)

<module 'util.mt' from '/mmfs1/gscratch/zlab/ahai/repo/infogain/util/mt.py'>

In [330]:
print(mt.post_proc(tokenizer.decode(mt.make_demo_prompt(src_points, tgt_points, 'finnish', 'english', tokenizer))))


 finnish : Kiertoajelut ovat halvempia suurille ryhmille, joten jos liikut yksin tai vain yhden ystävän kanssa, koeta tutustua muihin ihmisiin ja muodostaa 4–6 hengen ryhmä parempaa yksilöhintaa varten. english : Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 finnish : Vaatimukset on tarkoitettu tekemään maiden välisestä muuttovirrasta järjestelmällisen. english : These requirements are designed to provide an organized migratory flow between both countries.
 finnish : Varo: pikkukaupunkien baarit eivät täälläpäin aina ole parhaita ajanviettopaikkoja vieraspaikkakuntalaisille. english : Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.



In [331]:
def make_uncond_demo_prompt(tgt_points, tokenizer):
    uncond_demo_prompt_ids = [2]
    for t in tgt_points:
      uncond_demo_prompt_ids.extend(mt.xglm_tokenize(t['sentence'], tokenizer))
      uncond_demo_prompt_ids.append(2)
    return uncond_demo_prompt_ids

In [333]:
print(mt.post_proc(tokenizer.decode(make_uncond_demo_prompt(tgt_points, tokenizer))))


 Tours are cheaper for larger groups, so if you're by yourself or with just one friend, try to meet other people and form a group of four to six for a better per-person rate.
 These requirements are designed to provide an organized migratory flow between both countries.
 Beware: small-town bars here are not always good places for the out-of-state visitor to hang out.



In [44]:
load_dataset("gsarti/flores_101", 'mydfdsa')


/gscratch/zlab/ahai/miniconda3/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


ValueError: BuilderConfig mydfdsa not found. Available: ['afr', 'amh', 'ara', 'hye', 'asm', 'ast', 'azj', 'bel', 'ben', 'bos', 'bul', 'mya', 'cat', 'ceb', 'zho_simpl', 'zho_trad', 'hrv', 'ces', 'dan', 'nld', 'eng', 'est', 'tgl', 'fin', 'fra', 'ful', 'glg', 'lug', 'kat', 'deu', 'ell', 'guj', 'hau', 'heb', 'hin', 'hun', 'isl', 'ibo', 'ind', 'gle', 'ita', 'jpn', 'jav', 'kea', 'kam', 'kan', 'kaz', 'khm', 'kor', 'kir', 'lao', 'lav', 'lin', 'lit', 'luo', 'ltz', 'mkd', 'msa', 'mal', 'mlt', 'mri', 'mar', 'mon', 'npi', 'nso', 'nob', 'nya', 'oci', 'ory', 'orm', 'pus', 'fas', 'pol', 'por', 'pan', 'ron', 'rus', 'srp', 'sna', 'snd', 'slk', 'slv', 'som', 'ckb', 'spa', 'swh', 'swe', 'tgk', 'tam', 'tel', 'tha', 'tur', 'ukr', 'umb', 'urd', 'uzb', 'vie', 'cym', 'wol', 'xho', 'yor', 'zul', 'all']

In [87]:
langs =  ['afr', 'amh', 'ara', 'hye', 'asm', 'ast', 'azj', 'bel', 'ben', 'bos', 'bul', 'mya', 'cat', 'ceb', 'zho_simpl', 'zho_trad', 'hrv', 'ces', 'dan', 'nld', 'eng', 'est', 'tgl', 'fin', 'fra', 'ful', 'glg', 'lug', 'kat', 'deu', 'ell', 'guj', 'hau', 'heb', 'hin', 'hun', 'isl', 'ibo', 'ind', 'gle', 'ita', 'jpn', 'jav', 'kea', 'kam', 'kan', 'kaz', 'khm', 'kor', 'kir', 'lao', 'lav', 'lin', 'lit', 'luo', 'ltz', 'mkd', 'msa', 'mal', 'mlt', 'mri', 'mar', 'mon', 'npi', 'nso', 'nob', 'nya', 'oci', 'ory', 'orm', 'pus', 'fas', 'pol', 'por', 'pan', 'ron', 'rus', 'srp', 'sna', 'snd', 'slk', 'slv', 'som', 'ckb', 'spa', 'swh', 'swe', 'tgk', 'tam', 'tel', 'tha', 'tur', 'ukr', 'umb', 'urd', 'uzb', 'vie', 'cym', 'wol', 'xho', 'yor', 'zul']
for lang in langs:
  data = load_dataset("gsarti/flores_101", lang)
  with open(f'ref/101/txt/{lang}.ref', 'w') as out:
    for datum in data['dev']:
      out.write(f'{datum["sentence"]}\n')

Found cached dataset flores_101 (/gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/afr/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335)


  0%|          | 0/2 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/997 [00:00<?, ? examples/s]

Generating devtest split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Dataset flores_101 downloaded and prepared to /gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/amh/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335. Subsequent calls will reuse this data.


...

  0%|          | 0/2 [00:00<?, ?it/s]

Generating dev split:   0%|          | 0/997 [00:00<?, ? examples/s]

Generating devtest split:   0%|          | 0/1012 [00:00<?, ? examples/s]

Dataset flores_101 downloaded and prepared to /gscratch/zlab/ahai/hf/datasets/gsarti___flores_101/zul/1.0.0/e663cc717b274f2cef5142df786973abbf1966a7115d99509c062936d77d4335. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [88]:
import subprocess
langs =  ['afr', 'amh', 'ara', 'hye', 'asm', 'ast', 'azj', 'bel', 'ben', 'bos', 'bul', 'mya', 'cat', 'ceb', 'zho_simpl', 'zho_trad', 'hrv', 'ces', 'dan', 'nld', 'eng', 'est', 'tgl', 'fin', 'fra', 'ful', 'glg', 'lug', 'kat', 'deu', 'ell', 'guj', 'hau', 'heb', 'hin', 'hun', 'isl', 'ibo', 'ind', 'gle', 'ita', 'jpn', 'jav', 'kea', 'kam', 'kan', 'kaz', 'khm', 'kor', 'kir', 'lao', 'lav', 'lin', 'lit', 'luo', 'ltz', 'mkd', 'msa', 'mal', 'mlt', 'mri', 'mar', 'mon', 'npi', 'nso', 'nob', 'nya', 'oci', 'ory', 'orm', 'pus', 'fas', 'pol', 'por', 'pan', 'ron', 'rus', 'srp', 'sna', 'snd', 'slk', 'slv', 'som', 'ckb', 'spa', 'swh', 'swe', 'tgk', 'tam', 'tel', 'tha', 'tur', 'ukr', 'umb', 'urd', 'uzb', 'vie', 'cym', 'wol', 'xho', 'yor', 'zul']
for lang in langs:
  txt_path = f'ref/101/txt/{lang}.ref'
  spm_path = f'ref/101/spm/{lang}.ref.spm'
  cmd = f'python ../fairseq/scripts/spm_encode.py --model ../flores/flores200sacrebleuspm --output_format=piece --inputs {txt_path} --outputs {spm_path}'
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
  output, error = process.communicate()
  print(output)
  print(error)
  print('='*20)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


skipped 0 empty lines
filtered 0 lines


b''
None
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


skipped 0 empty lines
filtered 0 lines


b''
None
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

skipped 0 empty lines
filtered 0 lines


b''
None
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


skipped 0 empty lines
filtered 0 lines


b''
None
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
b''
None


skipped 0 empty lines
filtered 0 lines
